# 1. Data collection

In [2]:
import requests
import urllib.request
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
import os

In [ ]:
url = "https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")


## 1.1. Get the list of movies

In [1]:
movieslist = []

for tr in soup.findAll('tr'):
    for td in tr.findAll('td'):
        if "http" in (td.text):
            movieslist.append(td.text)
        

## 1.2. Crawl Wikipedia

In [ ]:
for i in range(0,10000):
    time.sleep(3)
    response = urlopen(movieslist[i])
    namefile = "article_" + str(i)+ ".html"
    with open(namefile, "wb") as f:
        f.write(response.read())
        f.close()

## 1.3 Parse downloaded pages

In [3]:
from bs4 import BeautifulSoup
import os
import csv

In [ ]:
##  read .HTML files
folder_dir = os.listdir('Html-data')
for i in range(1,10000):
    htmlfile=open(r'Html-data/article_'+str(i)+'.html','r',encoding='utf8')
    soup=BeautifulSoup(htmlfile,'html.parser')
    
    #Find Title of movies
    tsv_content=[]
    dict_content={}
    dict_content['title']=soup.select_one('h1',{"class":"firstheading:"}).text 
    
    # Find intro of movies
    
    pr_content = soup.select('p')
    try:
        dict_content['intro']="".join(pr_content[0].text.strip("\n"))
    except:
        dict_content['intro']="NA"
    
    # Find plot of movies
    try:
        dict_content['plot']="".join(pr_content[1].text.strip("\n"))
    except:
        dict_content['plot']="NA"
        
    for data in dict_content:
        tsv_content.append(dict_content[data])
        
    # Find infobox content 
    table_body=soup.find('tbody')
    if table_body!= None:
        allrows = table_body.find_all('tr')
        for rows in allrows:
            if rows.th!= None:
                if rows.th.text == 'Directed by':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Produced by':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Written by':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Starring':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Music by':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Release date':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Running time':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Country':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text == 'Language':
                    tsv_content.append("".join(rows.td.text.strip("\n")))
                elif rows.th.text =='Budget':
                    tsv_content.append("".join(rows.td.text.strip("\n"))) 
                
    # info saved into different .tsv files  
    
    with open('TSV-data/article_'+str(i)+'.tsv', 'wt') as out_file:
        try:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(tsv_content)
        except:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow('This page not found')

# 2. Search Engine

In [4]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

### Creating a function to clean all the data

In [8]:
def filtering(string):
    
    sentence1 = str(string).lower() # converting all in lower character to don't have double characters
    sentence2 = ''.join([i for i in sentence1 if not i.isdigit()]) # removing all the numbers
    
    # removing stopwords
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence2)
    no_stopwords = [w for w in word_tokens if not w in stop_words]

    # removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    no_punctuation = tokenizer.tokenize(str(no_stopwords))

    # stemming data
    ps = PorterStemmer() 
    stemming = []  
    for w in no_punctuation:
        stemming.append(ps.stem(w))
    stemmed_data = list(set(stemming)) # taking a set to take single word only 1 time
    
    return stemmed_data 

In [6]:
query = input()  # example

Love by the light of the moon


In [7]:
filtering(query) # example

['love', 'light', 'moon']

## 2.1. Conjunctive query


In [15]:
# Now we'll take all the words from intro and plot of each document. In tsv files we already have only intro and plot

### Creating a dictionary to put all the words from the tsv files (10.000 - 20.000)

In [9]:
path = os.getcwd()
d = []

for i in range(10000, 20000):
    try:
        with open(path+"\\tsvfile\\article_"+str(i)+".tsv", newline="", encoding="ISO-8859-1") as tsv_file:
                # Loop through each line of the file 
            for line in tsv_file:
                # Removing the leading spaces and newline character 
                line = line.strip() 
                
                # Convert the characters in line to  
                # lowercase to avoid case mismatch 
                line = line.lower() 

                # Split the line into words 
                words = line.split(" ") 
                d.append(line)
                    
    except FileNotFoundError:
            continue

### Now I assigne a variable "data" that contain in a list all the words filtered by the function filtering() created previously.


In [11]:
data = filtering("".join(d))  # in this way I get all the words in a single string and then I apply the filter at this string

In [12]:
len(data) # showing how many different words there are in the files from 10.000 to 20.000

119771

### Creating a vocabulary that maps each word to an integer

In [13]:
vocabulary = {k: v for v, k in enumerate(data)}